### Overview

Zonal Statistics is used to summarises the values of a raster dataset within the zones of a vector dataset. Here we select all Admin1 units of a country and calculate a sum of nighttime light pixel intensities over multiple years. This is a large computation that is enabled by cloud-hosted NightTime Lights data in the Cloud-Optimized GeoTIFF (COG) format and parallel computing on a local dask cluster.


### Setup and Data Download

The following blocks of code will install the required packages and download the datasets to your Colab environment.

In [ ]:
%%capture
if 'google.colab' in str(get_ipython()):
    !apt install libspatialindex-dev
    !pip install fiona shapely pyproj rtree
    !pip install geopandas
    !pip install rioxarray
    !pip install regionmask

In [ ]:
import os
import glob
import pandas as pd
import geopandas as gpd
import numpy as np
import xarray as xr
import rioxarray as rxr
import matplotlib.pyplot as plt
from datetime import datetime
import dask
import regionmask

In [ ]:
import warnings
import rasterio
warnings.filterwarnings("ignore", category=rasterio.RasterioDeprecationWarning)

In [ ]:
from dask.distributed import Client, progress
client = Client()  # set up local cluster on the machine
client

In [ ]:
dask.config.set({"array.slicing.split_large_chunks": False})
dask.config.set({'array.chunk-size': '32MiB'})

In [ ]:
data_folder = 'data'
output_folder = 'output'

if not os.path.exists(data_folder):
    os.mkdir(data_folder)
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

In [ ]:
def download(url):
    filename = os.path.join(data_folder, os.path.basename(url))
    if not os.path.exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

admin1_zipfile = 'ne_10m_admin_1_states_provinces.zip'
admin1_url = 'https://naciscdn.org/naturalearth/10m/cultural/'

download(admin1_url + admin1_zipfile)

### Data Pre-Processing

First we will read the GDL shapefile and filter to a country.
The 'adm1_code' column contains a unique id for all the counties present in the state, but it is of `object` type. We need to convert it to `int` type to be used in xarray.


In [ ]:
country_code = 'LK'
admin1_file_path = os.path.join(data_folder, admin1_zipfile)

admin1_df = gpd.read_file(admin1_file_path)

zones = admin1_df[admin1_df['iso_a2'] == country_code][['adm1_code', 'name', 'iso_a2', 'geometry']].copy()
zones['id'] = zones.reset_index().index + 1
zones

In [ ]:
bbox = zones.total_bounds
bbox

Next we read the NTL files and create an XArray object.

These files were download from [Harvard Dataverse](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/YGIVCD) and converted to Cloud-Optimized GeoTIFFs using GDAL.

 Example command `gdalwarp -of COG -co COMPRESS=DEFLATE -co PREDICTOR=2 -co NUM_THREADS=ALL_CPUS 2021_HasMask/LongNTL_2021.tif 2021.tif -te -180 -90 180 90 -dstnodata 0`
  
The resulting files are now hosted on a Google Cloud Storage bucket.

In [ ]:
start_year = 2015
end_year = 2020

In [ ]:

ntl_folder = 'https://storage.googleapis.com/spatialthoughts-public-data/ntl/npp_viirs_ntl'

da_list = []

for year in range(start_year, end_year + 1):
  cog_url = f'{ntl_folder}/{year}.tif'
  da = rxr.open_rasterio(
      cog_url,
      chunks=True).rio.clip_box(*bbox)
  dt = pd.to_datetime(year, format='%Y')
  da = da.assign_coords(time = dt)
  da = da.expand_dims(dim="time")
  da_list.append(da)

In [ ]:
ntl_datacube = xr.concat(da_list, dim='time').chunk('auto')
ntl_datacube

In [ ]:
ntl_datacube = ntl_datacube.sel(band=1, drop=True)

### Zonal Stats

Now we will extract the sum of the raster pixel values for every admin1 region in the selected countrey.

First, we need to convert the GeoDataFrame to a XArray Dataset. We will be using the `regionmask` module for that. It takes geodataframe, it's unique value as integer and converts the geodataframe into  a `xarray dataset` having dimension and coordinates same as of given input xarray dataset


In [ ]:
# Create mask of multiple regions from shapefile
mask = regionmask.mask_3D_geopandas(
        zones,
        ntl_datacube.x,
        ntl_datacube.y,
        drop=True,
        numbers="id",
        overlap=True
    )

In [ ]:
ntl_datacube = ntl_datacube.where(mask).chunk('auto')
ntl_datacube

In [ ]:
grouped = ntl_datacube.groupby('time').sum(['x','y'])
grouped

In [ ]:
%time grouped = grouped.compute()

In [ ]:
stats = grouped.drop('spatial_ref').to_dataframe('sum').reset_index()
stats

In [ ]:
stats['year'] = stats.time.dt.year
stats = stats.rename(columns={'region': 'id'})
stats

In [ ]:
results = zones.merge(stats, on='id')
results

Finally, we save the result to disk.

In [30]:
output = results[['adm1_code', 'name', 'iso_a2', 'year', 'sum']]
output

,adm1_code,name,iso_a2,year,sum
0,LKA-2454,Trikuṇāmalaya,LK,2015,449.778381
1,LKA-2454,Trikuṇāmalaya,LK,2016,453.339691
2,LKA-2454,Trikuṇāmalaya,LK,2017,640.339355
3,LKA-2454,Trikuṇāmalaya,LK,2018,656.189819
4,LKA-2454,Trikuṇāmalaya,LK,2019,854.694824
...,...,...,...,...,...
145,LKA-2448,Mahanuvara,LK,2016,856.860474
146,LKA-2448,Mahanuvara,LK,2017,1163.604858
147,LKA-2448,Mahanuvara,LK,2018,1226.470215
148,LKA-2448,Mahanuvara,LK,2019,1527.224609


In [ ]:
output_file = 'output.csv'
output_path = os.path.join(output_folder, output_file)

output.to_csv(output_path, index=False)
print('Successfully written output file at {}'.format(output_path))

### Exercise

Change the code to calculate the Zonal Statistics for all admin1 units in your chosen country.